In [ ]:
import numpy as np
import pandas as pd
import warnings
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
import random
import optuna
import lightgbm as lgb
warnings.filterwarnings('ignore')
import os
os.chdir(r"C:/Jupyter/229255_bus_riders_at_rush_hour_data")

from scipy import stats
from collections import Counter
from folium.plugins import MarkerCluster
from statsmodels.graphics.gofplots import qqplot
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
bts = pd.read_csv("bus_bts.csv")

train.shape, test.shape, bts.shape

In [ ]:
# 요일
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday

train.shape, test.shape

In [ ]:
# 요일
train['date'] = pd.to_datetime(train['date'])
train['weekday'] = train['date'].dt.weekday

test['date'] = pd.to_datetime(test['date'])
test['weekday'] = test['date'].dt.weekday

train.shape, test.shape

In [ ]:
# bus_route_id_weekday
# bus_route_id_weekday = bus_route_id + weekday
train['bus_route_id_weekday'] = train['bus_route_id'].astype(str) + ',' + train['weekday'].astype(str) 
test['bus_route_id_weekday'] = test['bus_route_id'].astype(str) + ',' + test['weekday'].astype(str) 

train.shape, test.shape

In [ ]:
# station_code_weekday
# station_code_weekday = station_code + weekday
train['station_code_weekday'] = train['station_code'].astype(str) + ',' + train['weekday'].astype(str)
test['station_code_weekday'] = test['station_code'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

In [ ]:
# route_station_weekday
# route_station_weekday = route_station + weekay
train['route_station_weekday'] = train['route_station'].astype(str) + ',' + train['weekday'].astype(str) 
test['route_station_weekday'] = test['route_station'].astype(str) + ',' + test['weekday'].astype(str)

train.shape, test.shape

In [ ]:
# on_time
# bts.csv 데이터에서 geton_time 열에서 시간대만 추출하여 on_time 컬럼을 만듬
bts['on_time']  = bts['geton_time'].apply(lambda x : x[:2])

bts.iloc[bts.query('on_time == "06"').index,13] = '6~7_ride'
bts.iloc[bts.query('on_time == "07"').index,13] = '7~8_ride'
bts.iloc[bts.query('on_time == "08"').index,13] = '8~9_ride'
bts.iloc[bts.query('on_time == "09"').index,13] = '9~10_ride'
bts.iloc[bts.query('on_time == "10"').index,13] = '10~11_ride'
bts.iloc[bts.query('on_time == "11"').index,13] = '11~12_ride'

bts.shape

In [ ]:
# 승 하차 시간대 통합 변수 (t ~ t+2)
# t~t+1, t+1~t+2 시간대 승하차인원을 합하여 t~t+2 시간대 승하차인원 변수를 만듬
train['68a']=train['6~7_ride']+train['7~8_ride'] 
train['810a']=train['8~9_ride']+train['9~10_ride']
train['1012a']=train['10~11_ride']+train['11~12_ride']

train['68b']=train['6~7_takeoff']+train['7~8_takeoff'] 
train['810b']=train['8~9_takeoff']+train['9~10_takeoff']
train['1012b']=train['10~11_takeoff']+train['11~12_takeoff']

test['68a']=test['6~7_ride']+test['7~8_ride']
test['810a']=test['8~9_ride']+test['9~10_ride']
test['1012a']=test['10~11_ride']+test['11~12_ride']

test['68b']=test['6~7_takeoff']+test['7~8_takeoff']
test['810b']=test['8~9_takeoff']+test['9~10_takeoff']
test['1012b']=test['10~11_takeoff']+test['11~12_takeoff']

train.shape, test.shape

In [ ]:
# Make features by using target variable
# 우리가 최종적으로 예측해야할 것은 각 일자별(date), 버스 노선(bus_route_id) 상의 정류장(station_name)의 퇴근시간 하차인원(18~20_ride)이다.

# bus_route_id, station_name, weekday의 각 조합별 퇴근시간 하차인원(18~20_ride)의 여러 통계량을 구한 후 이를 train set, test set에 모두 적용한다."
# target 변수를 train, test set에 적용할 수 있는 이유는 우리가 예측해야할 id는 date, bus_rout_id, station_name으로 구성되어있기 때문이다. 즉, 각각의 노선, 정류장별로 공통적인 패턴이 존재할 수 있다.
# 이 과정에서 NA 값이 생기는 이유는 train set에 없는 bus_route_id, station_name이 존재하기 때문이다.
def id_statistic(ID, col1, col2):
    
    # mean, sum
    rs_mean = train.groupby([ID])['18~20_ride'].agg([(col1, 'mean')]).reset_index()
    rs_sum = train.groupby([ID])['18~20_ride'].agg([(col2, 'sum')]).reset_index()
    rs_mean_sum = pd.merge(rs_mean, rs_sum, on=ID)

    # merge
    tr = pd.merge(train, rs_mean_sum, how='left', on=ID)
    te = pd.merge(test, rs_mean_sum, how='left', on=ID)

    # na -> mean
    tr[col1] = tr[col1].fillna(tr[col1].mean())
    tr[col2] = tr[col2].fillna(tr[col2].mean())
    te[col1] = te[col1].fillna(tr[col1].mean())
    te[col2] = te[col2].fillna(tr[col2].mean())
    
    return tr, te

In [ ]:
train, test = id_statistic('route_station', '1820_rs_mean', '1820_rs_sum')

train.shape, test.shape

In [ ]:
def plot_id_statistics(tr, te, ID, col1, col2):
    # 평균과 합계 데이터 추출
    train_grouped = tr.groupby(ID).agg({col1: 'mean', col2: 'sum'}).reset_index()
    test_grouped = te.groupby(ID).agg({col1: 'mean', col2: 'sum'}).reset_index()

    # 시각화 설정
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('ID Statistics Visualization', fontsize=16)

    # Train 데이터 시각화
    sns.scatterplot(ax=axes[0, 0], x=train_grouped[ID], y=train_grouped[col1])
    axes[0, 0].set_title(f'Train {col1} Mean by {ID}')
    axes[0, 0].set_xlabel(ID)
    axes[0, 0].set_ylabel(col1)

    sns.scatterplot(ax=axes[0, 1], x=train_grouped[ID], y=train_grouped[col2])
    axes[0, 1].set_title(f'Train {col2} Sum by {ID}')
    axes[0, 1].set_xlabel(ID)
    axes[0, 1].set_ylabel(col2)

    # Test 데이터 시각화
    sns.scatterplot(ax=axes[1, 0], x=test_grouped[ID], y=test_grouped[col1])
    axes[1, 0].set_title(f'Test {col1} Mean by {ID}')
    axes[1, 0].set_xlabel(ID)
    axes[1, 0].set_ylabel(col1)

    sns.scatterplot(ax=axes[1, 1], x=test_grouped[ID], y=test_grouped[col2])
    axes[1, 1].set_title(f'Test {col2} Sum by {ID}')
    axes[1, 1].set_xlabel(ID)
    axes[1, 1].set_ylabel(col2)

    plt.tight_layout()
    plt.show()

plot_id_statistics(train, test, 'route_station', '1820_rs_mean', '1820_rs_sum')

In [ ]:
train, test = id_statistic('bus_route_id', '1820_r_mean', '1820_r_sum')

train.shape, test.shape

In [ ]:
def plot_scatter(train, test):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    fig.suptitle('Scatter Plot of bus_route_id vs 18~20_ride', fontsize=16)

    # Train 데이터 산점도
    sns.scatterplot(ax=axes[0], data=train, x='bus_route_id', y='1820_r_sum')
    axes[0].set_title('Train Data')
    axes[0].set_xlabel('Bus Route ID')
    axes[0].set_ylabel('18~20 Ride')

    # Test 데이터 산점도
    sns.scatterplot(ax=axes[1], data=test, x='bus_route_id', y='1820_r_sum')
    axes[1].set_title('Test Data')
    axes[1].set_xlabel('Bus Route ID')
    axes[1].set_ylabel('18~20 Ride')

    plt.tight_layout()
    plt.show()

plot_scatter(train, test)

In [ ]:
train, test = id_statistic('station_code', '1820_s_mean', '1820_s_sum')

train.shape, test.shape

In [ ]:
def mean_statistics():
    # bus_route_id_weekday 기반 평균 계산
    f = train.groupby(['bus_route_id_weekday'])['18~20_ride'].agg([('mean_bus_weekday_ride', 'mean')]).reset_index()
    tr = pd.merge(train, f, how='left', on='bus_route_id_weekday')
    te = pd.merge(test, f, how='left', on='bus_route_id_weekday')
    tr['mean_bus_weekday_ride'] = tr['mean_bus_weekday_ride'].fillna(f['mean_bus_weekday_ride'].mean())
    te['mean_bus_weekday_ride'] = te['mean_bus_weekday_ride'].fillna(f['mean_bus_weekday_ride'].mean())

    # station_code_weekday 기반 평균 계산
    f = train.groupby(['station_code_weekday'])['18~20_ride'].agg([('mean_station_weekday_ride', 'mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='station_code_weekday')
    te = pd.merge(te, f, how='left', on='station_code_weekday')
    tr['mean_station_weekday_ride'] = tr['mean_station_weekday_ride'].fillna(f['mean_station_weekday_ride'].mean())
    te['mean_station_weekday_ride'] = te['mean_station_weekday_ride'].fillna(f['mean_station_weekday_ride'].mean())

    # route_station_weekday 기반 평균 계산
    f = train.groupby(['route_station_weekday'])['18~20_ride'].agg([('mean_route_station_weekday_ride', 'mean')]).reset_index()
    tr = pd.merge(tr, f, how='left', on='route_station_weekday')
    te = pd.merge(te, f, how='left', on='route_station_weekday')
    tr['mean_route_station_weekday_ride'] = tr['mean_route_station_weekday_ride'].fillna(f['mean_route_station_weekday_ride'].mean())
    te['mean_route_station_weekday_ride'] = te['mean_route_station_weekday_ride'].fillna(f['mean_route_station_weekday_ride'].mean())

    return tr, te

In [ ]:
train, test = mean_statistics()

train.shape, test.shape

In [ ]:
def congestion():
    # 버스 노선별 총 승객 수 계산
    df = train.groupby(['bus_route_id'])['18~20_ride'].agg([('passenger', 'sum')]).reset_index()

    # 승객 수에 따른 혼잡도 레벨 지정
    def get_congestion_level(passenger):
        if passenger > 10000:
            return 7
        elif passenger > 5000:
            return 6
        elif passenger > 2000:
            return 5
        elif passenger > 700:
            return 4
        elif passenger > 200:
            return 3
        elif passenger > 50:
            return 2
        else:
            return 1

    df['congestion'] = df['passenger'].apply(get_congestion_level)
    df = df[['bus_route_id', 'congestion']]

    # 학습 데이터와 테스트 데이터에 혼잡도 정보 병합
    tr = pd.merge(train, df, how='left', on='bus_route_id')
    te = pd.merge(test, df, how='left', on='bus_route_id')

    # 테스트 데이터의 결측치 처리
    te['congestion'] = te['congestion'].fillna(df['congestion'].median())

    return tr, te

In [ ]:
train, test = congestion()

train.shape, test.shape

In [ ]:
# location
# location = latitude + longitude
train['location'] = train['latitude'].astype(str) + ',' + train['longitude'].astype(str)
test['location'] = test['latitude'].astype(str) + ',' + test['longitude'].astype(str)

train.shape, test.shape

In [ ]:
# make cue column
train['cue']=0
test['cue']=1

train.shape, test.shape

In [ ]:
def morning() :
    
    # merge
    data = pd.concat([train, test])
    
    a = data.groupby(['route_station'])['1012a'].agg({'sum', 'mean'}).reset_index()
    a.columns = ['route_station', '1012a_sum','1012a_mean']

    b = data.groupby(['route_station'])['1012b'].agg({'sum', 'mean'}).reset_index()
    b.columns = ['route_station', '1012b_sum','1012b_mean']
    b = b[['1012b_sum','1012b_mean']]

    c = data.groupby(['route_station'])['10~11_ride'].agg({'sum', 'mean'}).reset_index()
    c.columns = ['route_station', '10~11_ride_sum','10~11_ride_mean']
    c = c[['10~11_ride_sum','10~11_ride_mean']]

    d = data.groupby(['route_station'])['10~11_takeoff'].agg({'sum', 'mean'}).reset_index()
    d.columns = ['route_station', '10~11_takeoff_sum','10~11_takeoff_mean']
    d = d[['10~11_takeoff_sum','10~11_takeoff_mean']]

    e = data.groupby(['route_station'])['11~12_ride'].agg({'sum', 'mean'}).reset_index()
    e.columns = ['route_station', '11~12_ride_sum','11~12_ride_mean']
    e = e[['11~12_ride_sum','11~12_ride_mean']]

    f = data.groupby(['route_station'])['11~12_takeoff'].agg({'sum', 'mean'}).reset_index()
    f.columns = ['route_station', '11~12_takeoff_sum','11~12_takeoff_mean']
    f = f[['11~12_takeoff_sum','11~12_takeoff_mean']]

    g = data.groupby(['route_station'])['1820_r_mean'].agg({'sum', 'mean'}).reset_index()
    g.columns = ['route_station', '1820_r_mean_sum','1820_r_mean_mean']
    g = g[['1820_r_mean_sum','1820_r_mean_mean']]

    h = data.groupby(['route_station'])['1820_r_sum'].agg({'sum', 'mean'}).reset_index()
    h.columns = ['route_station', '1820_r_sum_sum','1820_r_sum_mean']
    h = h[['1820_r_sum_sum','1820_r_sum_mean']]

    i = data.groupby(['route_station'])['1820_rs_mean'].agg({'sum', 'mean'}).reset_index()
    i.columns = ['route_station', '1820_rs_mean_sum','1820_rs_mean_mean']
    i = i[['1820_rs_mean_sum','1820_rs_mean_mean']]

    j = data.groupby(['route_station'])['1820_rs_sum'].agg({'sum', 'mean'}).reset_index()
    j.columns = ['route_station', '1820_rs_sum_sum','1820_rs_sum_mean']
    j = j[['1820_rs_sum_sum','1820_rs_sum_mean']]

    k = data.groupby(['route_station'])['1820_s_mean'].agg({'sum', 'mean'}).reset_index()
    k.columns = ['route_station', '1820_s_mean_sum','1820_s_mean_mean']
    k = k[['1820_s_mean_sum','1820_s_mean_mean']]

    l = data.groupby(['route_station'])['1820_s_sum'].agg({'sum', 'mean'}).reset_index()
    l.columns = ['route_station', '1820_s_sum_sum','1820_s_sum_mean']
    l = l[['1820_s_sum_sum','1820_s_sum_mean']]

    m = data.groupby(['route_station'])['1820_w_mean'].agg({'sum', 'mean'}).reset_index()
    m.columns = ['route_station', '1820_w_mean_sum','1820_w_mean_mean']
    m = m[['1820_w_mean_sum','1820_w_mean_mean']]

    n = data.groupby(['route_station'])['1820_w_sum'].agg({'sum', 'mean'}).reset_index()
    n.columns = ['route_station', '1820_w_sum_sum','1820_w_sum_mean']
    n = n[['1820_w_sum_sum','1820_w_sum_mean']]

    o = data.groupby(['route_station'])['68a'].agg({'sum', 'mean'}).reset_index()
    o.columns = ['route_station', '68a_sum','68a_mean']
    o = o[['68a_sum','68a_mean']]

    p = data.groupby(['route_station'])['68b'].agg({'sum', 'mean'}).reset_index()
    p.columns = ['route_station', '68b_sum','68b_mean']
    p = p[['68b_sum','68b_mean']]

    q = data.groupby(['route_station'])['6~7_ride'].agg({'sum', 'mean'}).reset_index()
    q.columns = ['route_station', '6~7_ride_sum','6~7_ride_mean']
    q = q[['6~7_ride_sum','6~7_ride_mean']]

    r = data.groupby(['route_station'])['6~7_takeoff'].agg({'sum', 'mean'}).reset_index()
    r.columns = ['route_station', '6~7_takeoff_sum','6~7_takeoff_mean']
    r = r[['6~7_takeoff_sum','6~7_takeoff_mean']]

    s = data.groupby(['route_station'])['7~8_ride'].agg({'sum', 'mean'}).reset_index()
    s.columns = ['route_station', '7~8_ride_sum','7~8_ride_mean']
    s = s[['7~8_ride_sum','7~8_ride_mean']]

    t = data.groupby(['route_station'])['7~8_takeoff'].agg({'sum', 'mean'}).reset_index()
    t.columns = ['route_station', '7~8_takeoff_sum','7~8_takeoff_mean']
    t = t[['7~8_takeoff_sum','7~8_takeoff_mean']]

    u = data.groupby(['route_station'])['810a'].agg({'sum', 'mean'}).reset_index()
    u.columns = ['route_station', '810a_sum','810a_mean']
    u = u[['810a_sum','810a_mean']]

    v = data.groupby(['route_station'])['810b'].agg({'sum', 'mean'}).reset_index()
    v.columns = ['route_station', '810b_sum','810b_mean']
    v = v[['810b_sum','810b_mean']]

    w = data.groupby(['route_station'])['8~9_ride'].agg({'sum', 'mean'}).reset_index()
    w.columns = ['route_station', '8~9_ride_sum','8~9_ride_mean']
    w = w[['8~9_ride_sum','8~9_ride_mean']]

    x = data.groupby(['route_station'])['8~9_takeoff'].agg({'sum', 'mean'}).reset_index()
    x.columns = ['route_station', '8~9_takeoff_sum','8~9_takeoff_mean']
    x = x[['8~9_takeoff_sum','8~9_takeoff_mean']]

    y = data.groupby(['route_station'])['9~10_ride'].agg({'sum', 'mean'}).reset_index()
    y.columns = ['route_station', '9~10_ride_sum','9~10_ride_mean']
    y = y[['9~10_ride_sum','9~10_ride_mean']]

    z = data.groupby(['route_station'])['9~10_takeoff'].agg({'sum', 'mean'}).reset_index()
    z.columns = ['route_station', '9~10_takeoff_sum','9~10_takeoff_mean']
    z = z[['9~10_takeoff_sum','9~10_takeoff_mean']]
    
    df = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z],axis=1)
    df = pd.merge(data, df, how='left', on='route_station')
    
    return df

In [ ]:
data = morning()

data.shape

In [ ]:
# 배차 간격
train['bus_route_id'] = train['bus_route_id'].astype(np.int64)
test['bus_route_id'] = test['bus_route_id'].astype(np.int64)

bts['geton_time2'] = pd.to_datetime(bts['geton_time'])

f = bts.groupby(['geton_date','geton_time2','geton_station_code','bus_route_id'])['user_count'].\
agg([('탑승객_수','sum')]).reset_index().\
sort_values(by=['geton_date','geton_station_code','bus_route_id','geton_time2'], ascending=True).reset_index()

f['index'] = list(range(0,len(f)))

In [ ]:
# time = []

# for i in range(0,len(f)-1):

#     if ((f.iloc[i].geton_date == f.iloc[i+1].geton_date) &\
#         (f.iloc[i].geton_station_code == f.iloc[i+1].geton_station_code) &\
#         (f.iloc[i].bus_route_id == f.iloc[i+1].bus_route_id)):

#         time.append(f.iloc[i+1].geton_time2 - f.iloc[i].geton_time2)

#     else:
#         time.append(0)

# time.insert(0, '0')

In [ ]:
# def get_sec(time_str):

#     h, m, s = time_str.split(':')

#     return int(h) * 3600 + int(m) * 60 + int(s)

In [ ]:
# def bus_interval() :
#     f['time'] = time
#     f['time2'] = f['time'].astype(str).str[7:]

#     interval = f.copy()
#     interval['time2'] = interval['time2'].astype(str).replace('','00:00:00')
#     interval['bus_route_id'] = interval['bus_route_id'].astype(object)

#     time4 = []

#     for i in interval['time2'] :
#         time4.append(get_sec(i))

#     interval['time4'] = time4
#     interval['time4'] = (interval['time4'] / 60).astype(int)

#     interval = interval[interval['time4'] > 3] # 간격이 3분보다 작은 것 제외
#     interval = interval[interval['time4'] < 180] # 간격이 3시간보다 큰 것 제외

#     interval = interval.groupby('bus_route_id')['time4'].agg([('bus_interval', 'mean')]).reset_index()
#     interval['bus_interval'] = interval['bus_interval'].astype(int)

#     # 나중에 시간을 절약하기 위해 csv 파일로 저장
#     interval.to_csv('bus_interval_final.csv', index = False)

#     print('success!')

In [ ]:
# bus_interval()

In [ ]:
bus_interval = pd.read_csv("bus_interval_final.csv")

In [ ]:
data['bus_route_id'] = data['bus_route_id'].astype(np.int64)

data['bus_route_id'] = data['bus_route_id'].astype(np.int64)

data['bus_interval'] = data['bus_interval'].fillna(9999)

In [ ]:
labelencoder = LabelEncoder()
df_encode = data[['bus_route_id','station_code', 'route_station_weekday', 'route_station']]
df_encoded = df_encode.apply(labelencoder.fit_transform)

data['bus_route_id2']=df_encoded['bus_route_id']
data['station_code2']=df_encoded['station_code']
data['route_station_weekday2']=df_encoded['route_station_weekday']
data['route_station2']=df_encoded['route_station']

data.shape

In [ ]:
data['in_out'].value_counts()
data['in_out'] = data['in_out'].map({'시내':0,'시외':1})

data.shape

In [ ]:
# from geopy.distance import geodesic

coords_jejusi = (33.500770, 126.522761) #제주시의 위도 경도
data['dis_jejusi'] = [geodesic((data['latitude'].iloc[i], data['longitude'].iloc[i]), coords_jejusi).km for i in range(len(data))]

coords_jejusicheong1 = (33.49892, 126.53035) #제주시청(광양방면)의 위도 경도
coords_jejuairport = (33.50661, 126.49345) #제주국제공항(구제주방면)의 위도 경도
coords_hallahosp = (33.48963, 126.486) #한라병원의 위도 경도
coords_rotary = (33.49143, 126.49678) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = (33.48902, 126.5392) #제주중앙여자고등학교의 위도 경도
coords_jejumarket = (33.51315, 126.52706) #동문시장의 위도 경도
coords_jejusclass = (33.47626, 126.48141) #제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = (33.51073, 126.5239) #중앙로(국민은행)의 위도 경도
coords_fiveway = (33.48667, 126.48092) # 노형오거리의 위도 경도
coords_law = (33.49363, 126.53476) # 제주지방법원(광양방면)의 위도 경도

data['dis_jejusicheong1'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusicheong1).km for i in range(len(data))]
data['dis_jejuairport'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejuairport).km for i in range(len(data))]
data['dis_hallahosp'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_hallahosp).km for i in range(len(data))]
data['dis_rotary'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_rotary).km for i in range(len(data))]
data['dis_jejucenterhigh'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejucenterhigh).km for i in range(len(data))]
data['dis_jejumarket'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejumarket).km for i in range(len(data))]
data['dis_jejusclass'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_jejusclass).km for i in range(len(data))]
data['dis_centerroad'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_centerroad).km for i in range(len(data))]
data['dis_fiveway'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_fiveway).km for i in range(len(data))]
data['dis_law'] = [geodesic((data['latitude'].iloc[i],data['longitude'].iloc[i]), coords_law).km for i in range(len(data))]

data.shape

In [ ]:
# 출근 시간의 총 승객 수
data['ride_sum'] = data['6~7_ride'] + data['7~8_ride'] + data['8~9_ride'] + data['9~10_ride'] + data['10~11_ride'] + data['11~12_ride'] 
data['takeoff_sum'] = data['6~7_takeoff'] + data['7~8_takeoff'] + data['8~9_takeoff'] + data['9~10_takeoff'] + data['10~11_takeoff'] + data['11~12_takeoff'] 

data.shape

In [ ]:
# 날짜 및 시간대 별 총 승객수
f = data.groupby('date')['6~7_ride'].agg([('6~7_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['7~8_ride'].agg([('7~8_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['8~9_ride'].agg([('8~9_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['9~10_ride'].agg([('9~10_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

f = data.groupby('date')['10~11_ride'].agg([('10~11_all_ride_number', 'sum')]).reset_index()
data = pd.merge(data, f, how='left')

data.shape

In [ ]:
# 주말, 주중
def h(x):
    if x ==5:
        return 1
    elif x==6:
        return 1
    else:
        return 0
data['weekend'] = data['weekday'].apply(h)

data.shape

In [ ]:
# 연휴
def g(x):
    if x in ['2019-09-12','2019-09-13','2019-09-14','2019-10-03','2019-10-09']:
        return 1
    else:
        return 0

data['holiday'] = data['date'].apply(g) 

data.shape

In [ ]:
# 요일 별 평균 승객 수
def week_mean() :

    df = data.reset_index(drop=True)
    df.groupby('weekday')['18~20_ride'].mean()
    df['weekdaymean']= 1

    index0 = df.query('weekday==0').index
    index1 = df.query('weekday==1').index
    index2 = df.query('weekday==2').index
    index3 = df.query('weekday==3').index
    index4 = df.query('weekday==4').index
    index5 = df.query('weekday==5').index
    index6 = df.query('weekday==6').index

    # 일별 18~20_ride 평균
    df.iloc[index0,-1] = 1.343710
    df.iloc[index1,-1] = 1.375319
    df.iloc[index2,-1] = 1.430856
    df.iloc[index3,-1] = 1.256710
    df.iloc[index4,-1] = 1.067439
    df.iloc[index5,-1] = 1.062123
    df.iloc[index6,-1] = 1.034282

    return df

In [ ]:
data = week_mean()

data.shape

In [ ]:
# 시내 및 시외버스 별 평균 탑승 승객
data['in_out_mean'] = 1
inindex = data.query('in_out == "시내"').index
outindex = data.query('in_out == "시외"').index

data.iloc[inindex,-1] = 1.228499
data.iloc[outindex,-1] = 2.044345
data['congestion'] = data['congestion'].astype('int64')

data.shape

In [ ]:
# 측정소와 정류장 사이 거리 계산
def dist() :
    jeju=(33.51411, 126.52969) # 제주 측정소 근처
    gosan=(33.29382, 126.16283) #고산 측정소 근처
    seongsan=(33.38677, 126.8802) #성산 측정소 근처
    po=(33.24616, 126.5653) #서귀포 측정소 근처

    t1 = [geodesic( (i,j), jeju).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t2 = [geodesic( (i,j), gosan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t3 = [geodesic( (i,j), seongsan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t4 = [geodesic( (i,j), po).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]

    data['dis_jeju'] = t1
    data['dis_gosan']=t2
    data['dis_seongsan']=t3
    data['dis_po']=t4

    total = pd.DataFrame(list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
    data['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)
    
    return data
data = dist()

data.shape

In [ ]:
data['dist_name']

In [ ]:
rain3 = pd.read_csv("rain3.csv")

# train, test의 변수명과 통일시키고, NaN의 값은 0.0000으로 변경
rain3 = rain3.rename(columns={"일시":"date","지점":"dist_name"})
rain3 = rain3.fillna(0.00000)
rain3['date'] = pd.to_datetime(rain3['date'])

rain3.columns

In [ ]:
print(data['dist_name'].dtypes)

In [ ]:
data['dist_name']

In [ ]:
print(rain3['dist_name'].dtypes)

In [ ]:
def dist() :
    jeju=(33.51411, 126.52969) # 제주 측정소 근처
    gosan=(33.29382, 126.16283) #고산 측정소 근처
    seongsan=(33.38677, 126.8802) #성산 측정소 근처
    po=(33.24616, 126.5653) #서귀포 측정소 근처

    t1 = [geodesic( (i,j), jeju).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t2 = [geodesic( (i,j), gosan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t3 = [geodesic( (i,j), seongsan).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]
    t4 = [geodesic( (i,j), po).km for i,j in list( zip( data['latitude'],data['longitude'] )) ]

    data['dis_jeju'] = t1
    data['dis_gosan']=t2
    data['dis_seongsan']=t3
    data['dis_po']=t4

    total = pd.DataFrame(list(zip( t1,t2,t3,t4)),columns=['jeju','gosan','seongsan','po'] )
    data['dist_name'] = total.apply(lambda x: x.argmin(), axis=1)
rain3['dist_name']
data['dist_name']

In [ ]:
# 숫자형으로 변환
rain3['dist_name'] = rain3['dist_name'].str.replace('gosan', '0')
rain3['dist_name'] = rain3['dist_name'].str.replace('seongsan', '1')
rain3['dist_name'] = rain3['dist_name'].str.replace('jeju', '2')
rain3['dist_name'] = rain3['dist_name'].str.replace('po', '3')
rain3['dist_name'] = rain3['dist_name'].astype(int)

In [ ]:
print(rain3['dist_name'].dtypes)

In [ ]:
data = pd.merge(data, rain3, how='left',on=['dist_name','date'])
data = pd.get_dummies(data,columns=['dist_name'])
# data = pd.get_dummies(data,columns=['si'])

data.shape

In [ ]:
# rainy_day
# 비 오는날=1, 비 안오는 날=0
def f(x):
    if x == 0:
        return 0
    else:
        return 1

In [ ]:
data['rainy_day'] = data['강수량(mm)'].apply(f)

data.shape

In [ ]:
# 승 하차 시간대 통합 변수 (t ~ t+3)
# t~t+1, t+1~t+2, t+2~t+3 시간대 승하차인원을 합하여 t~t+3 시간대 승하차인원 변수를 만듬
data['69a'] = data['6~7_ride']+data['7~8_ride']+data['8~9_ride']
data['912a']=data['9~10_ride']+data['10~11_ride']+data['11~12_ride']

data['69b'] = data['6~7_takeoff']+data['7~8_takeoff']+data['8~9_takeoff']
data['912b'] = data['9~10_takeoff']+data['10~11_takeoff']+data['11~12_takeoff']

data.shape

In [ ]:
data.to_csv('data_sample.csv', index=False)

In [ ]:
import pandas as pd
data = pd.read_csv('data_sample.csv')

In [ ]:
data['cue']

In [ ]:
# Make dataset
# 'cue' 열의 값이 0인 행을 train_data로
train_data = data[data['cue'] == 0].reset_index(drop=True)
# 'cue' 열의 값이 1인 행을 test_data로
test_data = data[data['cue'] == 1].reset_index(drop=True)

# 데이터프레임의 크기 출력
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

# 인덱스 확인
print("Train data index:", train_data.index)
print("Test data index:", test_data.index)

In [ ]:
# 요약통계량을 통한 변수를 만드는 과정에서 NA가 발생한다.

# target variable을 분리한 후 데이터를 저장한다.

y_train = train_data[['18~20_ride']]
train_data.shape, test_data.shape, y_train.shape
train_data.columns

In [ ]:
input_var_0=['in_out','latitude','longitude','6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride',
           '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff',
           'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 
           'dis_jejusi', 'dis_jejusicheong1','dis_jejuairport','dis_hallahosp', 'dis_rotary','dis_jejucenterhigh',
           'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law',
           'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_rs_mean', '1820_r_mean', '1820_s_mean', 'congestion',
           'station_code2', 'bus_route_id2', '일강수_10', '현재일기_10', '체감온도_10',
           '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number', '9~10_all_ride_number', '10~11_all_ride_number',
           '1820_w_mean','in_out_mean','weekdaymean','adult','kids','teen','elder','adult_prop', 'kids_prop', 'teen_prop', 'elder_prop',
           'mean_job_majorc', 'mean_job_smallc', 'mean_job_public', 'mean_job_profession', 'mean_job_self',
           'mean_vehicle_own_rat', 'mean_avg_income', 'mean_med_income', 'mean_avg_spend', 
           'rate_job_majorc', 'rate_job_smallc', 'rate_job_public', 'rate_job_profession', 'rate_job_self', 
           'rate_vehicle_own_rat', 'rate_avg_income', 'rate_med_income','rate_avg_spend',
           'sum_job_majorc', 'sum_job_smallc', 'sum_job_public', 'sum_job_profession', 'sum_job_self', 
           'sum_vehicle_own_rat', 'sum_avg_income', 'sum_med_income','sum_avg_spend',
           '68a', '810a', '1012a', '68b', '810b', '1012b','69a','912a','69b','912b',
           'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', '기온(°C)', '강수량(mm)',
           'dist_name_gosan', 'dist_name_jeju', 'dist_name_po', 'dist_name_seongsan', 'si_서귀포시', 'si_제주시',
           'school', 'transfer', 'dong2', 'rainy_day']

## columns

In [ ]:
input_var1 = ['in_out', 'latitude', 'longitude', '6~7_ride', '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff',
 '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', '11~12_takeoff', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
 'weekday_5', 'weekday_6', 'dis_jejusi', 'dis_jejusicheong1', 'dis_jejuairport', 'dis_hallahosp', 'dis_rotary', 'dis_jejucenterhigh',
 'dis_jejumarket', 'dis_centerroad', 'dis_jejusclass', 'dis_fiveway', 'dis_law', 'weekend', 'holiday', 'ride_sum', 'takeoff_sum', '1820_rs_mean',
 '1820_r_mean', '1820_s_mean', 'congestion', 'station_code2', 'bus_route_id2', '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number',
 '9~10_all_ride_number', '10~11_all_ride_number', b'1820_w_mean', 'in_out_mean', 'weekdaymean', 'adult', 'kids', 'teen', 'elder', '68a', '810a',
 '1012a', '68b', '810b', '1012b', '69a', '912a', '69b', '912b', 'dis_jeju', 'dis_gosan', 'dis_seongsan', 'dis_po', '기온(°C)', '강수량(mm)',
 'rainy_day']

In [ ]:
input_var3 = ['68a', '810a', 'bus_interval', 'dis_jejuairport', 'ride_sum', 'takeoff_sum', '1820_rs_mean', '1820_rs_sum', '1820_r_mean',
 '1820_r_sum', '1820_s_mean', '1820_s_sum', 'congestion', 'bus_route_id2', '6~7_all_ride_number', '7~8_all_ride_number', '8~9_all_ride_number',
 '1012a_mean', '1012b_sum', '10~11_ride_sum', '10~11_takeoff_sum', '11~12_ride_sum', '11~12_takeoff_sum', '1820_r_mean_sum', '1820_r_mean_mean',
 '1820_r_sum_sum', '1820_r_sum_mean', '1820_rs_mean_sum', '1820_s_mean_sum', '1820_s_mean_mean', '1820_s_sum_sum', '1820_s_sum_mean',
 '1820_w_mean_sum', '1820_w_mean_mean', '1820_w_sum_mean', '68a_sum', '68a_mean', '68b_sum', 'in_out', 'latitude', 'longitude', '6~7_ride',
 '7~8_ride', '8~9_ride', '9~10_ride', '10~11_ride', '11~12_ride', '6~7_takeoff', '7~8_takeoff', '8~9_takeoff', '9~10_takeoff', '10~11_takeoff', 
 '11~12_takeoff', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'dis_jejusi', '68b_mean', '6~7_ride_sum',
 '6~7_ride_mean', '6~7_takeoff_sum', '6~7_takeoff_mean', '7~8_ride_sum', '7~8_ride_mean', '7~8_takeoff_sum', '7~8_takeoff_mean', '810a_sum',
 '810b_sum', '8~9_ride_sum', '8~9_takeoff_sum', '8~9_takeoff_mean', '9~10_ride_sum', '9~10_takeoff_sum', 'route_station_weekday2']

# LightGBM을 사용한 Feature Selection

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train[input_var1], y_train, test_size=0.3, random_state= 123)

model = lgb.LGBMRegressor()
model.fit(X_train1, y_train1)

# feature importance 계산
importance = model.feature_importances_
feature_importance = pd.DataFrame({'feature': input_var1, 'importance': importance})
feature_importance = feature_importance.sort_values('importance', ascending=True)

# 교차 검증을 위한 RMSE 스코어러 정의
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# 중요도가 낮은 피처부터 하나씩 제거하면서 모델 성능 평가
rmse_list = []
features_list = []

for i in range(len(feature_importance)):
    selected_features = feature_importance['feature'][i:].tolist()
    X_train_selected = X_train1[selected_features]
    
    # 교차 검증을 사용하여 모델 성능 평가
    scores = cross_val_score(model, X_train_selected, y_train1, cv=5, scoring=rmse_scorer)
    mean_rmse = np.mean(scores)
    
    rmse_list.append(mean_rmse)
    features_list.append(selected_features)
    
    print(f"Removed {i} features, Mean RMSE: {mean_rmse}")

# 최적의 피처 조합 찾기
min_rmse_index = np.argmin(rmse_list)
best_features = features_list[min_rmse_index]
best_rmse = rmse_list[min_rmse_index]

print(f"\nBest Mean RMSE: {best_rmse} with features: {best_features}")

# 최적의 피처 조합 시각화
plt.figure(figsize=(10, 6))
plt.plot(range(len(rmse_list)), rmse_list, marker='o')
plt.xlabel('Number of features removed')
plt.ylabel('Mean RMSE')
plt.title('Mean RMSE vs Number of features removed')
plt.show()

In [ ]:
# 성능을 높이기 위해 중요도가 가장 높은 피처 1개 복구
best_var1 = best_features + ['elder']

# input_var3은 더이상 성능을 높일 수 없기에 Feature Selection을 진행 X
best_var3 = input_var3.copy()

## best_var1으로 학습한 LightGBM (GridSearchCV)

In [ ]:
lgbm = lgb.LGBMRegressor()

param_grid = {
    'n_estimators': [500],
    'num_leaves': [31, 50],
    'learning_rate': [0.05],
    'min_child_samples': [20, 50],
    'max_depth': [-1, 10],
    'feature_fraction': [0.8, 1.0],
    'bagging_fraction': [0.8, 1.0],
    'lambda_l1': [0, 0.1],
    'lambda_l2': [0, 0.1],
    'boosting_type': ['dart'],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator = lgbm, param_grid = param_grid, cv = 3, scoring = 'neg_mean_squared_error', 
                           verbose = 1, n_jobs = -1)

grid_search.fit(X_train[best_var1], y_train)

print("Best parameters : ", grid_search.best_params_)

best_model = grid_search.best_estimator_
X_test['18~20_ride'] = best_model.predict(X_test[best_var1])
X_test[['id','18~20_ride']].to_csv("lgb_gridCV_best_var1.csv",index=False)

## best_var1으로 학습한 LightGBM (Optuna)

In [ ]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 500,
        "verbosity": -1,
        "boosting_type": "dart",
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 31, 50),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 50),
        "lambda_l1": trial.suggest_float("lambda_l1", 0, 0.1),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 0.1),
        "max_depth": trial.suggest_int("max_depth", -1, 10)
    }
    
    model = lgb.LGBMRegressor(**params)

    scores = cross_val_score(model, X_train[best_var1], y_train, cv=5, scoring='neg_mean_squared_error')
    
    # 평균 RMSE 반환 (최소화하기 위해 - 붙임)
    return -scores.mean()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials = 100)  # 최적화 시도 횟수 100

print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

best_model = lgb.LGBMRegressor(**study.best_params)
best_model.fit(X_train[best_var1], y_train)
X_test['18~20_ride'] = best_model.predict(X_test[best_var1])
X_test[['id','18~20_ride']].to_csv("lgb_optuna_best_var1.csv",index=False)

## best_var3으로 학습한 LightGBM (GridSearchCV)

In [ ]:
lgbm = lgb.LGBMRegressor()

param_grid = {
    'n_estimators': [500],
    'num_leaves': [31, 50],
    'learning_rate': [0.05],
    'min_child_samples': [20, 50],
    'max_depth': [-1, 10],
    'feature_fraction': [0.8, 1.0],
    'bagging_fraction': [0.8, 1.0],
    'lambda_l1': [0, 0.1],
    'lambda_l2': [0, 0.1],
    'boosting_type': ['dart'],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search = GridSearchCV(estimator = lgbm, param_grid = param_grid, cv = 3, scoring = 'neg_mean_squared_error', 
                           verbose = 1, n_jobs = -1)

grid_search.fit(X_train[best_var3], y_train)

print("Best parameters : ", grid_search.best_params_)

best_model = grid_search.best_estimator_
X_test['18~20_ride'] = best_model.predict(X_test[best_var3])
X_test[['id','18~20_ride']].to_csv("lgb_gridCV_best_var3.csv",index=False)

## best_var3으로 학습한 LightGBM (Optuna)

In [ ]:
def objective(trial):
    params = {
        "objective": "regression",
        "metric": "rmse",
        "n_estimators": 500,
        "verbosity": -1,
        "boosting_type": "dart",
        "bagging_freq": 1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 31, 50),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.8, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 50),
        "lambda_l1": trial.suggest_float("lambda_l1", 0, 0.1),
        "lambda_l2": trial.suggest_float("lambda_l2", 0, 0.1),
        "max_depth": trial.suggest_int("max_depth", -1, 10)
    }
    
    model = lgb.LGBMRegressor(**params)

    scores = cross_val_score(model, X_train[best_var3], y_train, cv=5, scoring='neg_mean_squared_error')
    
    # 평균 RMSE 반환 (최소화하기 위해 - 붙임)
    return -scores.mean()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials = 100)  # 최적화 시도 횟수 100

print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

best_model = lgb.LGBMRegressor(**study.best_params)
best_model.fit(X_train[best_var3], y_train)
X_test['18~20_ride'] = best_model.predict(X_test[best_var3])
X_test[['id','18~20_ride']].to_csv("lgb_optuna_best_var3.csv",index=False)

## best_var1으로 학습한 RandomForest (GridSearchCV)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=200, max_depth = 10, min_samples_leaf = 4, min_samples_split = 10, max_features = 'auto',random_state=123)
rf_model.fit(X_train[best_var1], y_train)

X_test['18~20_ride'] = rf_model.predict(X_test[best_var1])
X_test[['id','18~20_ride']].to_csv("rf_gridCV_best_var1.csv",index=False)

## best_var1으로 학습한 RandomForest (Optuna)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=200, max_depth = 10, min_samples_leaf = 2, min_samples_split = 3, max_features = 'auto',random_state=123)
rf_model.fit(X_train[best_var1], y_train)

X_test['18~20_ride'] = rf_model.predict(X_test[best_var1])
X_test[['id','18~20_ride']].to_csv("rf_optuna_best_var1.csv",index=False)

## best_var3으로 학습한 RandomForest (GridSearchCV)

In [ ]:
rf_model = RandomForestRegressor(n_estimators = 200, max_depth = 10, min_samples_leaf = 1, min_samples_split = 2, max_features = 'sqrt',random_state=123)
rf_model.fit(X_train[best_var3], y_train)

X_test['18~20_ride'] = rf_model.predict(X_test[best_var3])
X_test[['id','18~20_ride']].to_csv("rf_gridCV_best_var3.csv",index=False)

## best_var3으로 학습한 RandomForest (Optuna)

In [ ]:
rf_model = RandomForestRegressor(n_estimators = 167, max_depth = 9, min_samples_leaf = 2, min_samples_split = 8, max_features = 'auto',random_state=123)
rf_model.fit(X_train[best_var3], y_train)

X_test['18~20_ride'] = rf_model.predict(X_test[best_var3])
X_test[['id','18~20_ride']].to_csv("rf_gridCV_best_var3.csv",index=False)

## 각 파일 concat 해 평균 값 구하고 dacon에 제출하여 rmse 값 확인하기

In [ ]:
import pandas as pd

In [ ]:
A=pd.read_csv("lgb_gridCV_best_var1.csv")
B=pd.read_csv("rf_gridCV_best_var1.csv")
C= pd.read_csv("lgb_gridCV_best_var3.csv")
D=pd.read_csv("rf_gridCV_best_var3.csv")


In [ ]:
#A+B 
combined = pd.concat([A, B])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()

result.to_csv("A+B.csv",index=False)

#A+C
combined = pd.concat([A, C])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("A+C.csv",index=False)

#A+D
combined = pd.concat([A, D])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("A+D.csv",index=False)

#B+C
combined = pd.concat([B, C])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("B+C.csv",index=False)

# B+D
# 데이터 합치기
combined = pd.concat([B, D])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("B+D.csv",index=False)

#C+D
combined = pd.concat([C, D])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("C+D.csv",index=False)

#A+B+C
combined = pd.concat([A,B,C])

# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("A+B+C.csv",index=False)

#A+B+D
combined = pd.concat([A,B,D])
# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("A+B+D.csv",index=False)

#B+C+D
combined = pd.concat([B,C,D])
# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("B+C+D.csv",index=False)

#A+C+D
combined = pd.concat([A,C,D])
# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("A+C+D.csv",index=False)

#A+B+C+D
combined = pd.concat([A,B,C,D])
# id별로 그룹화하고 평균 계산
result = combined.groupby('id')['18~20_ride'].mean().reset_index()
result.to_csv("A+B+C+D.csv",index=False)